In [0]:
print("Hello Spark!")

In [0]:
# DataFrames

In [0]:
emp_data = [
    (100, 'Eshwar', 20000),
    (101, 'Gautham', 25000),
    (102, 'Harish', 33000),
    (103, 'Nithin', 42000)
]

emp_cols = ["EmpId", "EmpName", "Salary"]

emp_df = spark.createDataFrame(data=emp_data, schema=emp_cols)
display(emp_df)

Databricks data profile. Run in Databricks to view.

In [0]:
emp_df.show()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


stu_data = [
    (100, 'Vishnu', 1, 'A'),
    (101, 'Ravi', 2, 'B'),
    (102, 'Rajesh', 3, 'C'),
    (103, 'Nithin', 3, 'C'),
    (104, 'Kishore', 2, 'B'),
]


schema = StructType([
    StructField('StuId', IntegerType(), True),
    StructField('StuName', StringType(), True),
    StructField('Class', IntegerType(), True),
    StructField('Section', StringType(), True)
])


stu_df = spark.createDataFrame(data=stu_data, schema=schema)
display(stu_df)

In [0]:
stu_df.rdd.getNumPartitions()

In [0]:
1 GB = 1024 MB
1024/128

In [0]:
1024/128

In [0]:
df = spark.table("samples.nyctaxi.trips")
df.count()

In [0]:
df.rdd.getNumPartitions()

In [0]:
from pyspark.sql import functions as F

def partition_memory_usage(iterator):
    import psutil
    import sys
    rows = list(iterator)
    mem = psutil.Process().memory_info().rss
    yield (rows[0]['partition_id'] if rows else None, len(rows), mem)

partitions_df = df.withColumn("partition_id", F.spark_partition_id())
partitioned = partitions_df.groupBy("partition_id").agg(
    F.count("*").alias("record_count")
)

partition_mem_df = partitions_df.rdd.mapPartitions(partition_memory_usage, preservesPartitioning=True) \
    .toDF(["partition_id", "record_count", "memory_bytes"])

result = partitioned.join(partition_mem_df, "partition_id", "left") \
    .drop(partitioned.record_count) \
    .withColumn("memory_MB", F.col("memory_bytes") / 1024 / 1024)

display(result)